#  Stage 05 — Interactive Visual Analytics (Folium)

# Overview

In this stage, we extend the exploratory analysis of Falcon 9 launch data by introducing interactive geospatial visualizations with Folium.
While previous analyses (using Matplotlib and Seaborn) highlighted correlations between payload, orbit, and launch outcomes, here we focus on the geographical dimension — specifically, how launch site location and its proximities may influence mission success.

Understanding these spatial patterns is valuable because the choice of launch site is a strategic decision, influenced by trajectory paths, recovery feasibility, and environmental considerations.

---

# Objectives

This lab explores launch sites interactively through the following tasks:

* Map Launch Sites — Plot all SpaceX launch locations on an interactive map.

* Overlay Outcomes — Visualize successful and failed launches per site.

* Distance Analysis — Compute distances between each launch site and its nearby features (e.g., coastline, cities).

By completing these tasks, we aim to reveal geospatial insights that complement earlier statistical findings.

## Set Up

In [ ]:
import folium
import pandas as pd

In [8]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Mark Launch Sites on a Map

The first step is to visualize the geographic distribution of SpaceX launch sites.
Using each site’s latitude and longitude, we can place them on an interactive Folium map to establish a baseline view of the infrastructure.

We will use the dataset spacex_launch_geo.csv, an augmented version of the launch records that includes geospatial attributes for each site.

In [ ]:
# Download and read the `spacex_launch_geo.csv`
spacex_df=pd.read_csv('spacex_launch_geo.csv')

Coordinates for each site:


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

# Visualizing Launch Coordinates

Raw latitude and longitude values are not very intuitive to interpret on their own. Instead of reading geographic coordinates as numbers, we can make them more meaningful by plotting them on a map.

To start, we create a Folium map object centered at the NASA Johnson Space Center (Houston, Texas). This will serve as our initial reference point before adding markers for each launch site.

In [11]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

`folium.Circle` is used to add a highlighted circle area with a text label on a specific coordinate.

In [12]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:blue;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

You see a circle near the city of Houston and you can zoom-in to see a larger circle.


## Adding Launch Site Markers

Next, we enhance the map by marking each launch site in the dataset launch_sites. To make the visualization more informative, we will use two elements:

folium.Circle — to highlight the area around each site.

folium.Marker — to label the site with its name.

This combination allows us to both locate and identify each launch site directly on the interactive map.

In [13]:
# Iterate over each launch site
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
for idx, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    circle = folium.Circle(
        location=coordinate,
        radius=1000,
        color='#000000', 
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))
    
    marker = folium.map.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )
    
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

The launch sites are marked in orange on the west coast near Los Angeles and the east coast near Florida. So they are not in proxomity of the equator line but all very close to the coast. 

# Success/failed launches for each site on the map

The map is enhanced by adding the launch outcomes for each site, to see which sites have high success rates. The "Class" colum from our data will provide insight.

In [ ]:
spacex_df.tail(10)

## Visualizing Launch Records

To enrich the map, we now add markers for individual launch records:

Green markers indicate successful launches (class = 1).

Red markers indicate failed launches (class = 0).

Since launches occur at only four distinct sites, many records share identical coordinates. Plotting them directly would clutter the map. To address this, we use marker clustering, which groups overlapping markers into expandable clusters for better readability.

We begin by creating a MarkerCluster object to manage these groups.

In [15]:
marker_cluster = MarkerCluster()

Dataframe called `marker_color` to store the marker colors based on the `class` value:


In [16]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

For each launch result in `spacex_df` data frame,  `folium.Marker` to `marker_cluster` added


In [17]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Launch Site: {record['Launch Site']}<br>Status: {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

site_map.add_child(marker_cluster)
site_map


# The distances between a launch site to its proximities

Add a `MousePosition` on the map to get coordinate for a mouse over a point on the map:

In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

## Distance between coasline and launch site

In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_coord = (28.5623, -80.5774)
coastline_coord = (28.56285, -80.56789)

# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline = calculate_distance(
    launch_site_coord[0], launch_site_coord[1],
    coastline_coord[0], coastline_coord[1]
)
distance_coastline

In [28]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)

site_map

## `PolyLine` between a launch site to the selected coastline point


In [34]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[launch_site_coord, coastline_coord], weight=1)
site_map.add_child(lines)

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


### Distance between City Point and launch site

In [35]:
city_point_coord = [28.61256, -80.80959]
distance_city = calculate_distance(launch_site_coord[0], launch_site_coord[1], city_point_coord[0], city_point_coord[1])

distance_marker = folium.Marker(
    location=city_point_coord,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#007849;"><b>{distance_city:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

line = folium.PolyLine(
    locations=[launch_site_coord, city_point_coord],
    weight=2,
    color='green',
    opacity=0.7
)
site_map.add_child(line)
site_map

## Distance between trainpoint and launch site

In [36]:
train_point_coord = [28.56428, -80.58673]
distance_train = calculate_distance(launch_site_coord[0], launch_site_coord[1], train_point_coord[0], train_point_coord[1])

distance_marker = folium.Marker(
    location=train_point_coord,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#007849;"><b>{distance_train:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

line = folium.PolyLine(
    locations=[launch_site_coord, train_point_coord],
    weight=2,
    color='blue',
    opacity=0.7
)
site_map.add_child(line)
site_map

## Distance between Highway point and launch site

In [37]:
high_point_coord = [28.56278, -80.57068]
distance_high = calculate_distance(launch_site_coord[0], launch_site_coord[1], high_point_coord[0], high_point_coord[1])

distance_marker = folium.Marker(
    location=high_point_coord,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#007849;"><b>{distance_high:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

line = folium.PolyLine(
    locations=[launch_site_coord, high_point_coord],
    weight=2,
    color='green',
    opacity=0.7
)
site_map.add_child(line)
site_map

## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
